## Setup

In [1]:
import os
from pathlib import Path
import sys

# If we're using Google Colab, we set the environment variable to point to the relevant folder in our Google Drive:
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    os.environ['SKIN_LESION_CLASSIFICATION'] = '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification'

# Otherwise, we use the environment variable on our local system:
project_environment_variable = "SKIN_LESION_CLASSIFICATION"

# Path to the root directory of the project:
project_path = Path(os.environ.get(project_environment_variable))

# Relative path to /scripts (from where custom modules will be imported):
scripts_path = project_path.joinpath("scripts")

# Add this path to sys.path so that Python will look there for modules:
sys.path.append(str(scripts_path))

# Now import path_step from our custom utils module to create a dictionary to all subdirectories in our root directory:
from utils import path_setup
path = path_setup.subfolders(project_path)

path['project'] : D:\projects\skin-lesion-classification
path['images'] : D:\projects\skin-lesion-classification\images
path['models'] : D:\projects\skin-lesion-classification\models
path['expository'] : D:\projects\skin-lesion-classification\expository
path['literature'] : D:\projects\skin-lesion-classification\literature
path['notebooks'] : D:\projects\skin-lesion-classification\notebooks
path['presentation'] : D:\projects\skin-lesion-classification\presentation
path['scripts'] : D:\projects\skin-lesion-classification\scripts


## Loading and preprocessing metadata

In [2]:
from typing import Type, Union      # For type hints
from processing import process      # Custom module for processing metadata

data_dir: Path = path["images"]     # Path to directory containing metadata.csv file
csv_filename: str = "metadata.csv"  # The filename
    
restrict_to: Union[dict, None] = None                   # Remove all records *unless* column k lies in list v, for k : v in restrict_to dictionary.    
remove_if: Union[dict, None] = None                     # Remove all records if column k lies in list v, for k : v in remove_if dictionary.    
drop_row_if_missing_value_in: Union[list, None] = None  # We drop all rows for which there is a missing value in a column from this list.   
                                    
tvr: int = 3              # Ratio of training set to validation set. See discussion below for explanation.
seed: int = 0             # Random seed for parts of the process where randomness is called for.
keep_first: bool = False  # If False, then, for each lesion, we choose a random image to assign to our training set. 
stratified: bool = True   # If True, we stratify classes so that the proportions remain as stable as possible after train/val split. 
                          # If False, the proportions will be roughly similar.

to_classify: Union[list, dict] = ["mel",   # These are the lesion types we are interested in classifying. 
                                  "bcc",   # Any missing ones will be grouped together as the 0-label class: no need to write "other" here.
                                  "akiec", # If 'other' is not desired, use restrict_to attribute above
                                  "nv",]

train_one_img_per_lesion: Union[bool, None] = False # If False, we take advantage of the (in some cases) multiple images of a lesion in our dataset
val_one_img_per_lesion: Union[bool, None] = False   # If False, we will validate our model by combining multiple predictions for a lesion (for multiple images of it) into a single prediction
val_expansion_factor: Union[int, None] = 3          # A random transformation may be applied to an image before making a prediction.
                                                    # For a given lesion, we may make multiple predictions (as specified here), and combine them into a single prediction.
    
sample_size: Union[None, dict] = {"mel": 2000,     # Handling class imbalance by upsampling minority classes/downsampling majority classes     
                                  "bcc": 2000,     # Specify how many images of each lesion diagnosis we want in our training set.
                                  "akiec": 2000, 
                                  "nv": 2000,
                                  "other" : 2000,} # Could also leave out "other" here, and include e.g. "df: 2000" if we wanted to.    

In [3]:
# Create an instance of the process class with attribute values as above.
metadata = process(data_dir=data_dir,
                   csv_filename=csv_filename,
                   restrict_to=restrict_to,
                   remove_if=remove_if,
                   drop_row_if_missing_value_in=drop_row_if_missing_value_in,
                   tvr=tvr,
                   seed=seed,
                   keep_first=keep_first,
                   stratified=stratified,
                   to_classify=to_classify,
                   train_one_img_per_lesion=train_one_img_per_lesion,
                   val_one_img_per_lesion=val_one_img_per_lesion,
                   val_expansion_factor=val_expansion_factor,
                   sample_size=sample_size,)

Successfully loaded file 'D:\projects\skin-lesion-classification\images\metadata.csv'.
Inserted 'num_images' column in dataframe, to the right of 'lesion_id' column.
Created self._label_dict (maps labels to indices).
Inserted 'label' column in dataframe, to the right of 'dx' column.
Added 'set' column to dataframe, with values 't1', 'v1', 'ta', and 'va', to the right of 'localization' column.
See self.df
Balancing classes in training set.
See self._df_balanced.
Expanding validation set: for each lesion in the validation set, 3 predictions will ultimately be combined into a single prediction.
See self._df_expanded_val.
Combining training dataframe and validation dataframe into a single dataframe.
See self.df_combined.
Creating a small sample dataframe for code testing.
See self._df_sample_batch.


## Train/val split

<!-- <details>
    <summary><b><i>Train test split explanation: click here to expand/collapse</i></b></summary> -->
    
We partition our dataset based on ```lesion_id```, **not** on ```image_id```: that way, every lesion will be represented in training or in validation, but not both.

For each classification task, we will train a model by making use of
* **exactly one** image for every lesion in our training set;
* **all** images of every lesion in our training set.

In both cases, we will vaildate our model by making use of 
* **exactly one** image for every lesion in our validation set;
* **all** images of every lesion in our validation set (at least, _potentially_ all of them). 

**However**, we will make only one prediction per lesion (```lesion_id```) in our validation set: if there are multiple images of a lesion in the validation set, we will combine the predictions for the multiple images into a single prediction for the lesion.

Accordingly, we proceed as follows. We'll explain by example, assuming the dataset is not filtered before splitting (if it is, the number of distinct lesions will be less than $7470$, and the proportions will be different).
1. Randomly select (without replacement) a proportion of our $7470$ distinct ```lesion_id```s and label them with ```t``` (train). 
2. Label the remaining ```lesion_id```s with ```v``` (validate).
3. For each ```lesion_id``` labeled with a ```t```:
    * Select an ```image_id``` and label it ```t1```.
    * Label all (if any) remaining ```image_id```s corresponding to this ```lesion_id``` with ```ta```.
4.  For each ```lesion_id``` labeled with a ```v```:
    * Select an ```image_id``` and label it ```v1```.
    * Label all (if any) remaining ```image_id```s corresponding to this ```lesion_id``` with ```va```.

In Step 1, the number of ```lesion_id```s randomly selected to be labeled ```t``` will be such that the ratio of ```t```s to ```v```s is as close as possible to a specified ratio ```tvr``` (we default to $3$, i.e. $\approx75\%$ of lesions are represented in training). In Steps 3 and 4, the first substep can be done randomly (our default choice), or we can simply choose the "first" image in our table that corresponds to the lesion (see ```keep_first``` attribute of the ```process``` class). 

The four train/val scenarios we could consider are:
* ```t1v1```: train on precisely those images labeled ```t1``` and validate on precisely those labeled ```v1```.
* ```t1va```: train on precisely those images labeled ```t1``` and validate on precisely those labeled ```v1``` **or** ```va```.
* ```tav1```: train on precisely those images labeled ```t1``` **or** ```ta``` and validate on precisely those labeled ```v1```.
* ```tava```: train on precisely those images labeled ```t1``` **or** ```ta``` and validate on precisely those labeled ```v1``` ***or*** ```va```.

The mnemonic is ```t``` for training, ```v``` for validation, ```1``` for one-image-per-lesion, and ```a``` for all images.
<!-- </details> -->

In [4]:
# Let's have a look at our metadata dataframe, which is now just an attribute of the metadata instance of the process class.
from utils import print_header

print_header("First five rows of metadata table")

to_print = ["Added columns\n".upper(), 
            "\'num_images\': number of images of lesion in dataset", 
            "\'label\': class to which lesion belongs",
            "\'set\': train/val assignment",
            "\'t*\': lesion is in the training set",
            "\'v*\': lesion is in the validation set",
            "\'ta\': we would train on this image if training a model on all images of each lesion in the training set",
            "\'t1\': we would train on this image if training a model either on exactly one image per lesion in the training set, or on all images for each lesion in the training set",
            "\'va': we could use this image as part of validation process, combining the predictions for all images of the corresponding lesion into a single prediction for that lesion" ,
            "\'v1': we would use this image for validation if validating a model either on all images per lesion in the validation set (combining them into a single prediction), or on exactly one image per lesion in the validation set",
           ]

print("\n- ".join(to_print))
display(metadata.df.head())


FIRST FIVE ROWS OF METADATA TABLE

ADDED COLUMNS

- 'num_images': number of images of lesion in dataset
- 'label': class to which lesion belongs
- 'set': train/val assignment
- 't*': lesion is in the training set
- 'v*': lesion is in the validation set
- 'ta': we would train on this image if training a model on all images of each lesion in the training set
- 't1': we would train on this image if training a model either on exactly one image per lesion in the training set, or on all images for each lesion in the training set
- 'va': we could use this image as part of validation process, combining the predictions for all images of the corresponding lesion into a single prediction for that lesion
- 'v1': we would use this image for validation if validating a model either on all images per lesion in the validation set (combining them into a single prediction), or on exactly one image per lesion in the validation set


,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta
1,HAM_0000118,2,ISIC_0025030,bkl,0,histo,80.0,male,scalp,t1
2,HAM_0002730,2,ISIC_0026769,bkl,0,histo,80.0,male,scalp,va
3,HAM_0002730,2,ISIC_0025661,bkl,0,histo,80.0,male,scalp,v1
4,HAM_0001466,2,ISIC_0031633,bkl,0,histo,75.0,male,ear,va


## Class distribution

In [5]:
for across in ["lesions", "images"]:
    for subset in ["all", "train", "val"]:
        process.dx_dist(metadata, subset = subset, across = across)


DISTRIBUTION OF LESIONS BY DIAGNOSIS: OVERALL



dx,nv,other,mel,bcc,akiec
freq,5403.00,898.00,614.00,327.00,228.00
%,72.33,12.02,8.22,4.38,3.05


Total lesions: 7470.


DISTRIBUTION OF LESIONS BY DIAGNOSIS: TRAIN



dx,nv,other,mel,bcc,akiec
freq,4052.00,673.00,460.00,245.00,171.00
%,72.34,12.02,8.21,4.37,3.05


Total lesions: 5601 (74.98% of all lesions).


DISTRIBUTION OF LESIONS BY DIAGNOSIS: VAL



...

Total images: 10015.


DISTRIBUTION OF IMAGES BY DIAGNOSIS: TRAIN



dx,nv,other,mel,bcc,akiec
freq,5007.00,1008.00,831.00,384.00,250.00
%,66.94,13.48,11.11,5.13,3.34


Total images: 7480 (74.69% of all images).


DISTRIBUTION OF IMAGES BY DIAGNOSIS: VAL



dx,nv,other,mel,bcc,akiec
freq,1698.00,348.00,282.00,130.00,77.00
%,66.98,13.73,11.12,5.13,3.04


Total images: 2535 (25.31% of all images).



## Balancing the training set

<!-- <details>
    <summary><b><i>Balancing/upsampling explanation: click here to expand/collapse</i></b></summary> -->

We explain the balancing procedure by way of example. (This is performed by the ```balance``` method of the ```process``` class in our ```processing``` module.) We assume the dataset has not been filtered, training to validation ratio is $3$, etc. There are $460$ distinct melanoma lesions represented in our training set. As most melanoma are represented by multiple distinct images, there are a total of $831$ distinct images of melanoma lesions in our training set. Suppose we want our training set to contain $2000$ melanoma images: each of the $460$ distinct melanoma lesions will be represented by $2000/460 \approx 4.35$ images on average. We do not merely sample with replacement.

The goal is to (a) have as little variance as possible in the number of times a lesion is represented, and (b) use as many distinct images as possible (taking advantage of the fact that there are multiple _distinct_ images of most melanoma). Thus, we note that $2000 = 4\times 460 + 160$, so we will use each of the $460$ distinct melanoma lesions four times, and make the remainder up by randomly sample $160$ distinct lesions from the $160$. In other words, exactly $300$ distinct lesions will each be represented by exactly four images, and exactly $160$ distinct lesions will each be represented by exactly five images: $2000 = 300 \times 4 + 160 \times 5$. 

How do we select the four images of each distinct melanoma lesion (plus another one image for $160$ of them)? Consider lesion id ```HAM_0000871``` for example: there are three distinct images of this lesion in our data set. Thus, if ```train_one_img_per_lesion``` is ```False```, we will use all three of them, and then randomly select one more (or two more if this particular lesion were to be one of the $160$ that are represented five times). See below. On the other hand, if ```train_one_img_per_lesion``` is ```True```, we have no choice but to use the one image (label ```t1```) four times.
    
<!-- </details> -->

In [6]:
# The specific numbers in this example assume a certain choice for the attributes, including 
# sample_size: Union[None, dict] = {"mel": 2000,         
#                                   "bcc": 2000, 
#                                   "akiec": 2000, 
#                                   "nv": 2000,
#                                   "other" : 2000,}

df_tmp = metadata._df_balanced

to_print = ["HAM_0000871 represented by four images\n".upper(),
            "Three distinct images of this lesion to choose from: ISIC_0025964, ISIC_0030623, and ISIC_0025964",
            "Use ISIC_0025964 once, ISIC_0030623 twice, and ISIC_0025964 once",]

print_header("Eg: Representations of lesion HAM_0000871 in balanced training set")

print("\n- ".join(to_print))

display(df_tmp[df_tmp['lesion_id'] == 'HAM_0000871'])


EG: REPRESENTATIONS OF LESION HAM_0000871 IN BALANCED TRAINING SET

HAM_0000871 REPRESENTED BY FOUR IMAGES

- Three distinct images of this lesion to choose from: ISIC_0025964, ISIC_0030623, and ISIC_0025964
- Use ISIC_0025964 once, ISIC_0030623 twice, and ISIC_0025964 once


,lesion_id,lesion_mult,num_images,image_id,img_mult,dx,label,dx_type,age,sex,localization,set
1773,HAM_0000871,4,3,ISIC_0025964,2,mel,3,histo,40.0,female,chest,ta
1774,HAM_0000871,4,3,ISIC_0025964,2,mel,3,histo,40.0,female,chest,ta
1775,HAM_0000871,4,3,ISIC_0030623,1,mel,3,histo,40.0,female,chest,t1
3088,HAM_0000871,4,3,ISIC_0026506,1,mel,3,histo,40.0,female,trunk,ta


In [7]:
# The specific numbers given in this example assume a certain choice for the attributes, including 
# sample_size: Union[None, dict] = {"mel": 2000,         
#                                   "bcc": 2000, 
#                                   "akiec": 2000, 
#                                   "nv": 2000,
#                                   "other" : 2000,}

df_tmp = df_tmp[df_tmp['set'].isin(["ta", "t1"]) & (df_tmp['dx'] == 'mel')]

print_header("Eg: Melanoma in balanced training set")

to_print = ["Value counts for \'lesion_mult\' column\n".upper(),
            "300 distinct melanoma lesions each represented by four images: 300*4 = 1200",
            "160 distinct melanoma lesions each represented by five images: 160*5 = 800",]

print("\n- ".join(to_print[:3]))
display(df_tmp['lesion_mult'].value_counts())

print("\n- ".join(to_print[3:]))

display(df_tmp)


EG: MELANOMA IN BALANCED TRAINING SET

VALUE COUNTS FOR 'LESION_MULT' COLUMN

- 300 distinct melanoma lesions each represented by four images: 300*4 = 1200
- 160 distinct melanoma lesions each represented by five images: 160*5 = 800


4    1200
5     800
Name: lesion_mult, dtype: int64

,lesion_id,lesion_mult,num_images,image_id,img_mult,dx,label,dx_type,age,sex,localization,set
1773,HAM_0000871,4,3,ISIC_0025964,2,mel,3,histo,40.0,female,chest,ta
1774,HAM_0000871,4,3,ISIC_0025964,2,mel,3,histo,40.0,female,chest,ta
1775,HAM_0000871,4,3,ISIC_0030623,1,mel,3,histo,40.0,female,chest,t1
1776,HAM_0000040,5,1,ISIC_0027190,5,mel,3,histo,80.0,male,upper extremity,t1
1777,HAM_0000040,5,1,ISIC_0027190,5,mel,3,histo,80.0,male,upper extremity,t1
...,...,...,...,...,...,...,...,...,...,...,...,...
7773,HAM_0002552,5,3,ISIC_0032936,2,mel,3,histo,25.0,male,upper extremity,ta
7774,HAM_0002552,5,3,ISIC_0032936,2,mel,3,histo,25.0,male,upper extremity,ta
7780,HAM_0002552,5,3,ISIC_0033232,1,mel,3,histo,25.0,male,upper extremity,ta
9998,HAM_0003521,5,2,ISIC_0032258,2,mel,3,histo,70.0,female,back,ta


## Expanding the validation set

As mentioned already, we make one prediction per lesion. However, we may have multiple images of a given lesion at our disposal: we could make a prediction for each of them and combine them somehow into a single prediction for the lesion. Even if there is only one image of a lesion, we could make multiple predictions on it: if a random transformation is applied to an image before our model makes a prediction on it, this would yield a different array of probabilities each time. Again, we could combine the results into a single prediction.

This is what the attribute ```val_expansion_factor``` of the ```process``` class is concerned with. Similarly to the way the boolean attribute ```train_one_img_per_lesion``` affects the balancing of the training set, ```val_one_img_per_lesion```, if ```True```, forces us to replicate one single image per lesion in the validation set as many times as specified by ```val_expansion_factor```; howevery, if ```val_one_img_per_lesion``` is ```False```, we may take advantage of other images of the lesion (if available).

In [8]:
# The specific numbers given in this example assume a certain choice for the attributes  

df_tmp = metadata._df_expanded_val

df_tmp = df_tmp[df_tmp['set'].isin(["va", "v1"]) & (df_tmp['dx'] == 'mel')]

print_header("Eg: Melanoma in validation set")

to_print = [f"- Note that \'lesion_mult\' is always {metadata.val_expansion_factor}",
            "HAM_0005678 represented by three images",
            "Two distinct images of this lesion to choose from: ISIC_0031023 and ISIC_0028086",
            "Use ISIC_0031023 once, and ISIC_0028086 twice",]

print("\n- ".join(to_print))

display(df_tmp)


EG: MELANOMA IN VALIDATION SET

- Note that 'lesion_mult' is always 3
- HAM_0005678 represented by three images
- Two distinct images of this lesion to choose from: ISIC_0031023 and ISIC_0028086
- Use ISIC_0031023 once, and ISIC_0028086 twice


,lesion_id,lesion_mult,num_images,image_id,img_mult,dx,label,dx_type,age,sex,localization,set
603,HAM_0005678,3,2,ISIC_0031023,1,mel,3,histo,60.0,male,chest,v1
604,HAM_0005678,3,2,ISIC_0028086,2,mel,3,histo,60.0,male,chest,va
605,HAM_0005678,3,2,ISIC_0028086,2,mel,3,histo,60.0,male,chest,va
606,HAM_0006722,3,2,ISIC_0030443,1,mel,3,histo,85.0,female,lower extremity,va
607,HAM_0006722,3,2,ISIC_0031499,2,mel,3,histo,85.0,female,lower extremity,v1
...,...,...,...,...,...,...,...,...,...,...,...,...
1060,HAM_0004746,3,2,ISIC_0029021,1,mel,3,histo,65.0,female,back,va
1061,HAM_0002525,3,2,ISIC_0025188,2,mel,3,histo,55.0,male,face,va
1062,HAM_0002525,3,2,ISIC_0025188,2,mel,3,histo,55.0,male,face,va
1063,HAM_0001953,3,2,ISIC_0025611,2,mel,3,histo,65.0,male,back,va


In [9]:
# The specific numbers given in this example assume a certain choice for the attributes  

# Now let's have a look at our balanced data + validation set

print_header("EG: First five rows of combined balanced training/expanded validation set")

to_print = ["Added columns\n".upper(), 
            "\'lesion_mult\': total number of (not necessarily distinct) images of lesion in this (balanced) training set", 
            "\'img_mult\': number of times this image is used in this (balanced) training set",
            "\'lesion_mult\' = sum \'img_mult\' over distinct \'image_id\' corresponding to \'lesion_id\'",
           ]

print("\n- ".join(to_print))

display(metadata.df_combined.head())

print_header("EG: Last two rows of balanced training set, and first three rows of expanded validation set")
display(metadata.df_combined.iloc[9998:10003])

print_header("EG: Last five rows of expanded validation set")
display(metadata.df_combined.tail())


EG: FIRST FIVE ROWS OF COMBINED BALANCED TRAINING/EXPANDED VALIDATION SET

ADDED COLUMNS

- 'lesion_mult': total number of (not necessarily distinct) images of lesion in this (balanced) training set
- 'img_mult': number of times this image is used in this (balanced) training set
- 'lesion_mult' = sum 'img_mult' over distinct 'image_id' corresponding to 'lesion_id'


,lesion_id,lesion_mult,num_images,image_id,img_mult,dx,label,dx_type,age,sex,localization,set
0,HAM_0000118,3,2,ISIC_0027419,1,bkl,0,histo,80.0,male,scalp,ta
1,HAM_0000118,3,2,ISIC_0025030,2,bkl,0,histo,80.0,male,scalp,t1
2,HAM_0000118,3,2,ISIC_0025030,2,bkl,0,histo,80.0,male,scalp,t1
3,HAM_0005132,3,2,ISIC_0025837,1,bkl,0,histo,70.0,female,back,t1
4,HAM_0005132,3,2,ISIC_0025209,2,bkl,0,histo,70.0,female,back,ta



EG: LAST TWO ROWS OF BALANCED TRAINING SET, AND FIRST THREE ROWS OF EXPANDED VALIDATION SET



,lesion_id,lesion_mult,num_images,image_id,img_mult,dx,label,dx_type,age,sex,localization,set
9998,HAM_0003521,5,2,ISIC_0032258,2,mel,3,histo,70.0,female,back,ta
9999,HAM_0003521,5,2,ISIC_0032258,2,mel,3,histo,70.0,female,back,ta
10000,HAM_0002730,3,2,ISIC_0026769,2,bkl,0,histo,80.0,male,scalp,va
10001,HAM_0002730,3,2,ISIC_0026769,2,bkl,0,histo,80.0,male,scalp,va
10002,HAM_0002730,3,2,ISIC_0025661,1,bkl,0,histo,80.0,male,scalp,v1



EG: LAST FIVE ROWS OF EXPANDED VALIDATION SET



,lesion_id,lesion_mult,num_images,image_id,img_mult,dx,label,dx_type,age,sex,localization,set
15602,HAM_0004282,3,3,ISIC_0033358,1,akiec,1,histo,65.0,female,face,va
15603,HAM_0004282,3,3,ISIC_0033151,1,akiec,1,histo,65.0,female,face,v1
15604,HAM_0004592,3,2,ISIC_0029141,2,akiec,1,histo,60.0,female,face,va
15605,HAM_0004592,3,2,ISIC_0029141,2,akiec,1,histo,60.0,female,face,va
15606,HAM_0005705,3,2,ISIC_0031430,1,akiec,1,histo,75.0,female,lower extremity,va


## Other attributes of the 'process' class

In [10]:
import pandas as pd
# There are some implicit attributes of our process class:
metadata_hidden_attributes = metadata.get_hidden_attributes()

for key, value in metadata_hidden_attributes.items():
    if type(value) != pd.DataFrame:
        print(f"{key} : {value}\n")
    else:
        print(f"{key}\n")

_csv_file_path : D:\projects\skin-lesion-classification\images\metadata.csv

_label_dict : {'df': 0, 'bkl': 0, 'vasc': 0, 'akiec': 1, 'bcc': 2, 'mel': 3, 'nv': 4}

_label_codes : {0: 'other', 1: 'akiec', 2: 'bcc', 3: 'mel', 4: 'nv'}

_num_labels : 7

_df_train1

_df_train_a

_df_val1

_df_val_a

_df_sample_batch

_df_balanced

_expanded_val_df

df_combined



## Fine-tuning ResNet18

### Transforms

In [11]:
# Now let's set values for the attributes of our resnet18 class (the model we will use with out processed data).
# One of the attributes has to do with image transformations.

import torchvision.transforms as transforms

transform = transforms.Compose([
    
transforms.RandomCrop((300, 300)),
transforms.Resize((224,224)), # Resize images to fit ResNet input size
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])    

In [30]:
import importlib

import multiclass_models

importlib.reload(multiclass_models)

<module 'multiclass_models' from 'D:\\projects\\skin-lesion-classification\\scripts\\multiclass_models.py'>

In [12]:
import pandas as pd
from typing import Union, List, Callable

source: process = metadata     
model_dir: Path = path["models"]                 # Path to directory where models/model info/model results are stored.
transform: Union[None, 
                 transforms.Compose, 
                 List[Callable]] = transform     # Transform to be applied to images before feeding to ResNet-18
batch_size: int = 32                             # Mini-batch size: default 32.
epochs: int = 10                                 # Number of epochs (all layers unfrozen from the start): default 10.
base_learning_rate: float = 0.001                # Learning rate to start with: default 0.001. Using Adam optimizer.
filename_stem: str = "rn18"                      # For saving model and related files. train set and num epochs will be appended automatically. Default "rn18mc".
filename_suffix: str = "demo"                    # Something descriptive and unique for future reference and to avoid over-writing other files. Default empty string "".
overwrite: Union[bool, None] = True              # If False, any will generate an unused filename for saving .pth, .csv files etc., but appending a two-digit number.
                                                 # If None, will default to False. Only set to True if confident that training done on previous instances with same filename stem and suffix can be over-written.
code_test: Union[bool, None] = True

In [13]:
# Create an instance of the resnet18 class with attribute values as above.
from multiclass_models import resnet18

resnet18_demo = resnet18(                                   
    source=source,                                           
    model_dir=model_dir,
    transform=transform,
    batch_size=batch_size,
    epochs=epochs,                                          
    base_learning_rate=base_learning_rate,
    filename_stem=filename_stem,
    filename_suffix=filename_suffix,                         
    overwrite=overwrite,
    code_test=code_test,                                     
)

Existing files will be overwritten. 
Base filename: rn18_tava_bal_test_1e_demo_00
Attributes saved to file: D:\projects\skin-lesion-classification\models\rn18_tava_bal_test_1e_demo_00_attributes.json


<details><summary>Click here to read about an error we found and corrected.</summary>
    
Below, we were catching ```RuntimeError: size mismatch (got input: [5], target: [1])```, before we incorporated one-hot encoding. This wasn't caught while testing on small batches of images, however: only after attempting to train on the entire training set on Google Colab.
    
Then, we caught a ```RuntimeError``` when we tested on dataset of size ```k``` (as in ```metadata.df.sample(n=k, random_state=metadata.seed)``` above) for these values of ```k```: ```1,12,13,14,15,16```. 

Sizes ```2,3,...,11,17,18,19,20,...,33``` were fine. Unfortunately, we were been thrown the same error when training on the full training set (on Google Colab) of size ~7,500, after a good 30 to 60 minutes. 

We thought it might be something to do with the size of the dataset module the batch size (```32```), but received no errors for sizes ```44,45```. Also, the number of epochs seemed irrelevant: changing number of epochs to ```1``` or ```2``` while keeping all else constant, had no effect as far as this error was concerned.

We noted that the error was thrown, at least in some cases (we did not re-check all sizes ```k``` again), not during the training loop, but during validation. We checked the validation sets ```resnet18mc_test._df_val``` that were being fed into the dataloader at this stage, and noticed that for all of the problematic ```k```, and none of the others, there was precisely one image in the validation set. We changed the line ```loss = criterion(outputs.squeeze(), labels)``` to ```loss = criterion(outputs, labels)``` in the validation loop, and found that the ```RuntimeError``` was no longer being thrown. We also made the same change to the training loop. We notice that, regardless of the use of the ```.squeeze()``` method, the ```outputs.shape``` seems to always be of the form ```torch.Size([m, 5])``` (when the code works), which is the right shape, making squeezing redundant.
</details>
<details><summary>Click here to see the error details.</summary>

```
    
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
Input In [83], in <cell line: 2>()
      1 # Train the model on the specified training data by calling the train method:
----> 2 resnet18mc_test.train()

File D:\projects\skin-lesion-classification\scripts\multiclass_models.py:195, in resnet18.train(self)
    193                     val_outputs = model(val_images)
    194 #                     val_loss = criterion(val_outputs.squeeze(), val_labels.long())
--> 195                     val_loss = criterion(val_outputs.squeeze(), val_labels)
    196                     if self.Print:
    197                         print(f"outputs.shape: {outputs.shape}")

File ~\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1511, in Module._wrapped_call_impl(self, *args, **kwargs)
   1509     return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]
   1510 else:
-> 1511     return self._call_impl(*args, **kwargs)

File ~\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1520, in Module._call_impl(self, *args, **kwargs)
   1515 # If we don't have any hooks, we want to skip the rest of the logic in
   1516 # this function, and just call forward.
   1517 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1518         or _global_backward_pre_hooks or _global_backward_hooks
   1519         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1520     return forward_call(*args, **kwargs)
   1522 try:
   1523     result = None

File ~\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:1179, in CrossEntropyLoss.forward(self, input, target)
   1178 def forward(self, input: Tensor, target: Tensor) -> Tensor:
-> 1179     return F.cross_entropy(input, target, weight=self.weight,
   1180                            ignore_index=self.ignore_index, reduction=self.reduction,
   1181                            label_smoothing=self.label_smoothing)

File ~\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:3059, in cross_entropy(input, target, weight, size_average, ignore_index, reduce, reduction, label_smoothing)
   3057 if size_average is not None or reduce is not None:
   3058     reduction = _Reduction.legacy_get_string(size_average, reduce)
-> 3059 return torch._C._nn.cross_entropy_loss(input, target, weight, _Reduction.get_enum(reduction), ignore_index, label_smoothing)

RuntimeError: 0D or 1D target tensor expected, multi-target not supported
```
</details>

### Code test with small batch of images

In [14]:
# from utils import print_header

instance = resnet18_demo

print_header("Code test: Training set")
print(f"{instance._df_train.shape[0]} images".upper())
display(instance._df_train.head())

print_header("Code test: Validation set")
print(f"{instance._df_val.shape[0]} images".upper())
display(instance._df_val.head())


CODE TEST: TRAINING SET

169 IMAGES


,lesion_id,lesion_mult,num_images,image_id,img_mult,dx,label,dx_type,age,sex,localization,set
0,HAM_0003768,3,2,ISIC_0029929,1,bkl,0,histo,80.0,male,upper extremity,t1
1,HAM_0004928,3,2,ISIC_0031424,1,bkl,0,histo,65.0,male,neck,t1
2,HAM_0004928,3,2,ISIC_0029770,2,bkl,0,histo,65.0,male,neck,ta
3,HAM_0004928,3,2,ISIC_0029770,2,bkl,0,histo,65.0,male,neck,ta
4,HAM_0003768,3,2,ISIC_0026634,2,bkl,0,histo,80.0,male,upper extremity,ta



CODE TEST: VALIDATION SET

42 IMAGES


,lesion_id,lesion_mult,num_images,image_id,img_mult,dx,label,dx_type,age,sex,localization,set
169,HAM_0003218,3,1,ISIC_0033305,3,bkl,0,consensus,75.0,male,back,v1
170,HAM_0003218,3,1,ISIC_0033305,3,bkl,0,consensus,75.0,male,back,v1
171,HAM_0003218,3,1,ISIC_0033305,3,bkl,0,consensus,75.0,male,back,v1
172,HAM_0000983,3,1,ISIC_0033490,3,bkl,0,consensus,NaN,unknown,unknown,v1
173,HAM_0000983,3,1,ISIC_0033490,3,bkl,0,consensus,NaN,unknown,unknown,v1


In [12]:
# Train the model on the specified training data by calling the train method:
# from utils import print_header

instance = resnet18_demo

print_header("Code test: training and validation")
instance.train()


CODE TEST: TRAINING AND VALIDATION

image_id, label, ohe-label: ISIC_0031526, 2, tensor([0., 0., 1., 0., 0.])
image_id, label, ohe-label: ISIC_0033551, 2, tensor([0., 0., 1., 0., 0.])
image_id, label, ohe-label: ISIC_0033551, 2, tensor([0., 0., 1., 0., 0.])
image_id, label, ohe-label: ISIC_0033975, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033975, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0029278, 2, tensor([0., 0., 1., 0., 0.])
image_id, label, ohe-label: ISIC_0027560, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0031831, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0029713, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0030344, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0031728, 2, tensor([0., 0., 1., 0., 0.])
image_id, label, ohe-label: ISIC_0030953, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0026138, 3, tensor([0., 0., 0., 1., 0.])
i

image_id, label, ohe-label: ISIC_0025302, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0026634, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033847, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0027598, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0025302, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0029278, 2, tensor([0., 0., 1., 0., 0.])
image_id, label, ohe-label: ISIC_0032329, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0031424, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0030953, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0027598, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0030230, 2, tensor([0., 0., 1., 0., 0.])
image_id, label, ohe-label: ISIC_0030280, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0029929, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_00253

In [ ]:
import json

# Sample dictionary with various value types
my_dict = {
    'str_key': 'Hello',
    'int_key': 42,
    'float_key': 3.14,
    'bool_key': True,
    'list_key': [1, 2, 3],
    'dict_key': {'a': 1, 'b': 2},
    'function_key': lambda x: x + 1,  # Ignored because it's not a valid type
    'dataframe_key': None  # Ignored because it's not a valid type
}

# Filter and convert values to strings
filtered_dict = {}
for key, value in my_dict.items():
    if isinstance(value, (str, int, float, bool, list, dict)):
        filtered_dict[key] = str(value)

# Save the filtered dictionary to a JSON file
with open('filtered_output.json', 'w') as json_file:
    json.dump(filtered_dict, json_file)


In [18]:
# Let's look at the training and validation loss for each epoch:
instance = resnet18mc_test
print_header("Code test")
print(f"Loss dictionary (TRAINING AND VALIDATION LOSS FROM EACH EPOCH)".upper())
display(instance.epoch_losses)


CODE TEST

LOSS DICTIONARY (TRAINING AND VALIDATION LOSS FROM EACH EPOCH)


{'train_loss': array([0.98603733]), 'val_loss': array([2.32553625])}

In [61]:
# The model will be saved as a .pth file in the directory given by model_dir attribute.
# Sans .pth extension, the filename is
instance = resnet18mc_test
print_header("Code test")
file_path = model_dir.joinpath(instance._filename + ".pth")
print(f"Model path: {file_path}")


CODE TEST

Model path: D:\projects\skin-lesion-classification\models\rn18mc_t1_1e_test_00.pth


<details><summary>Click here to read about an error encountered when using Google Colab's GPU for inference.</summary>
    
To fix, added line ```images = images.to(device)``` in definition of ```inference``` function:
    
```
with torch.no_grad():
    for images, labels, image_ids in dataloader:
        # Send input tensor to device
        images = images.to(device)
        # Make predictions using the model
        outputs = model(images)
        # Apply softmax to get probabilities                
        probabilities = softmax(outputs)
```

```
    
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-14-4e2245e9b2a7> in <cell line: 3>()
      1 # We can feed our entire dataframe through the trained model to obtain predictions for all lesions/images.
      2 # Data can be loaded from a pre-saved .pth file if it is not still in memory.
----> 3 inference_df = resnet18mc_test.inference()
      4 display(inference_df)

8 frames
/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/scripts/multiclass_models.py in inference(self, df_infer, filename, Print)
    258             for images, labels, image_ids in dataloader:
    259                 # Make predictions using the model
--> 260                 outputs = model(images)
    261                 # Apply softmax to get probabilities
    262                 probabilities = softmax(outputs)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in _wrapped_call_impl(self, *args, **kwargs)
   1509             return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]
   1510         else:
-> 1511             return self._call_impl(*args, **kwargs)
   1512 
   1513     def _call_impl(self, *args, **kwargs):

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in _call_impl(self, *args, **kwargs)
   1518                 or _global_backward_pre_hooks or _global_backward_hooks
   1519                 or _global_forward_hooks or _global_forward_pre_hooks):
-> 1520             return forward_call(*args, **kwargs)
   1521 
   1522         try:

/usr/local/lib/python3.10/dist-packages/torchvision/models/resnet.py in forward(self, x)
    283 
    284     def forward(self, x: Tensor) -> Tensor:
--> 285         return self._forward_impl(x)
    286 
    287 

/usr/local/lib/python3.10/dist-packages/torchvision/models/resnet.py in _forward_impl(self, x)
    266     def _forward_impl(self, x: Tensor) -> Tensor:
    267         # See note [TorchScript super()]
--> 268         x = self.conv1(x)
    269         x = self.bn1(x)
    270         x = self.relu(x)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in _wrapped_call_impl(self, *args, **kwargs)
   1509             return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]
   1510         else:
-> 1511             return self._call_impl(*args, **kwargs)
   1512 
   1513     def _call_impl(self, *args, **kwargs):

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in _call_impl(self, *args, **kwargs)
   1518                 or _global_backward_pre_hooks or _global_backward_hooks
   1519                 or _global_forward_hooks or _global_forward_pre_hooks):
-> 1520             return forward_call(*args, **kwargs)
   1521 
   1522         try:

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py in forward(self, input)
    458 
    459     def forward(self, input: Tensor) -> Tensor:
--> 460         return self._conv_forward(input, self.weight, self.bias)
    461 
    462 class Conv3d(_ConvNd):

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py in _conv_forward(self, input, weight, bias)
    454                             weight, bias, self.stride,
    455                             _pair(0), self.dilation, self.groups)
--> 456         return F.conv2d(input, weight, bias, self.stride,
    457                         self.padding, self.dilation, self.groups)
    458 

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor
    
```
</details>

<details><summary>Click here to read about a subsequent error encountered when using Google Colab's GPU for inference.</summary> The above fix did not break the inference process when a CPU was in use, however on the GPU... ```TypeError``` below was thrown. 
    
Fix: in definition of inference function, we went from   
    
```
# Apply softmax to get probabilities                
                probabilities = softmax(outputs)
        
                series_dict = { }
                series_dict["image_id"] = pd.Series(image_ids)

                for idx, label in enumerate(self.label_codes.values()):
                    series_dict["prob_" + label] = pd.Series(probabilities[:,idx])    
```   

to this:    
    
    
```
# Apply softmax to get probabilities                
                probabilities = softmax(outputs)
                
                # Move probabilities to CPU before converting to NumPy array
                probabilities_cpu = probabilities.cpu().numpy()
        
                series_dict = { }
                series_dict["image_id"] = pd.Series(image_ids)

                for idx, label in enumerate(self.label_codes.values()):
                    series_dict["prob_" + label] = pd.Series(probabilities_cpu[:,idx])    
```    
    
    
    
TypeError                                 Traceback (most recent call last)
<ipython-input-16-4e2245e9b2a7> in <cell line: 3>()
      1 # We can feed our entire dataframe through the trained model to obtain predictions for all lesions/images.
      2 # Data can be loaded from a pre-saved .pth file if it is not still in memory.
----> 3 inference_df = resnet18mc_test.inference()
      4 display(inference_df)

3 frames
/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/scripts/multiclass_models.py in inference(self, df_infer, filename, Print)
    268 
    269                 for idx, label in enumerate(self.label_codes.values()):
--> 270                     series_dict["prob_" + label] = pd.Series(probabilities[:,idx])
    271 
    272                 batch_df = pd.DataFrame(series_dict)

/usr/local/lib/python3.10/dist-packages/pandas/core/series.py in __init__(self, data, index, dtype, name, copy, fastpath)
    468                     data = data.copy()
    469             else:
--> 470                 data = sanitize_array(data, index, dtype, copy)
    471 
    472                 manager = get_option("mode.data_manager")

/usr/local/lib/python3.10/dist-packages/pandas/core/construction.py in sanitize_array(data, index, dtype, copy, raise_cast_failure, allow_2d)
    614         if hasattr(data, "__array__"):
    615             # e.g. dask array GH#38645
--> 616             data = np.array(data, copy=copy)
    617         else:
    618             data = list(data)

/usr/local/lib/python3.10/dist-packages/torch/_tensor.py in __array__(self, dtype)
   1060             return handle_torch_function(Tensor.__array__, (self,), self, dtype=dtype)
   1061         if dtype is None:
-> 1062             return self.numpy()
   1063         else:
   1064             return self.numpy().astype(dtype, copy=False)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.    
    
```

In [62]:
# We can feed our entire dataframe through the trained model to obtain predictions for all lesions/images.
# Data can be loaded from a pre-saved .pth file if it is not still in memory.
instance = resnet18mc_test
print_header("Code test: inference")
instance.inference(save=True);


CODE TEST: INFERENCE

image_id, label, ohe-label: ISIC_0027419, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0025030, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033482, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033201, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0032997, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033529, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0034303, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0034031, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033124, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033449, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033676, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0027652, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0031168, 0, tensor([1., 0., 0., 0., 0.])
image_id, label

image_id, label, ohe-label: ISIC_0029760, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0030555, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0031372, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0026313, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0034135, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033626, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033780, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0030060, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0030211, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0034048, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033905, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033981, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0031900, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_00310

In [70]:
instance = resnet18mc_test

print_header("Code test: appending model's probabilities to dataframe")

to_print = ["- Notice that the probabilities may vary with each execution of a prediction.",
            "This is because a random transformation may be applied to each image before our model makes a prediction on it."]

print("\n- ".join(to_print))

display(instance._inference_df.head())


CODE TEST: APPENDING MODEL'S PROBABILITIES TO DATAFRAME

- Notice that the probabilities may vary with each execution of a prediction.
- This is because a random transformation may be applied to each image before our model makes a prediction on it.


,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_bcc,prob_nv,prob_akiec
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta,0.00281,0.488154,0.320958,0.112851,0.075228
1,HAM_0000118,2,ISIC_0025030,bkl,0,histo,80.0,male,scalp,t1,0.006403,0.54659,0.207624,0.160286,0.079096
2,HAM_0006663,2,ISIC_0033482,bkl,0,histo,85.0,female,lower extremity,t1,0.001759,0.126734,0.534769,0.037566,0.299172
3,HAM_0006663,2,ISIC_0033201,bkl,0,histo,85.0,female,lower extremity,ta,0.005804,0.15067,0.624709,0.059166,0.159651
4,HAM_0003162,2,ISIC_0032997,bkl,0,histo,75.0,male,back,t1,0.058898,0.161691,0.408894,0.197394,0.173124


In [73]:
# Or we can make predictions for individual lesions/images:
instance = resnet18mc_test
print_header("Code test: prediction on individual lesion or image")

to_print = ["- Notice that the probabilities may vary with each execution of a prediction.",
            "This is because a random transformation may be applied to each image before our model makes a prediction on it."]

print("\n- ".join(to_print))
print("")

display(instance.prediction("HAM_0000118"))
display(instance.prediction("ISIC_0027419"))


CODE TEST: PREDICTION ON INDIVIDUAL LESION OR IMAGE

- Notice that the probabilities may vary with each execution of a prediction.
- This is because a random transformation may be applied to each image before our model makes a prediction on it.

image_id, label, ohe-label: ISIC_0027419, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0025030, 0, tensor([1., 0., 0., 0., 0.])


,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_bcc,prob_nv,prob_akiec
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta,0.008713,0.468143,0.250376,0.210331,0.062437
1,HAM_0000118,2,ISIC_0025030,bkl,0,histo,80.0,male,scalp,t1,0.004319,0.504687,0.22604,0.211857,0.053098


image_id, label, ohe-label: ISIC_0027419, 0, tensor([1., 0., 0., 0., 0.])


,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_bcc,prob_nv,prob_akiec
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta,0.001932,0.570881,0.270781,0.096323,0.060083


In [74]:
instance = resnet18mc_test
print_header("Code test: model state dictionary")
display(instance.state_dict)


CODE TEST: MODEL STATE DICTIONARY



OrderedDict([('conv1.weight',
              tensor([[[[-1.1318e-02, -7.4821e-03, -3.2353e-03,  ...,  5.4820e-02,
                          1.5258e-02, -1.4726e-02],
                        [ 1.0332e-02,  8.7299e-03, -1.1085e-01,  ..., -2.7214e-01,
                         -1.3038e-01,  2.4072e-03],
                        [-7.7705e-03,  5.8199e-02,  2.9471e-01,  ...,  5.1848e-01,
                          2.5500e-01,  6.2471e-02],
                        ...,
                        [-2.8479e-02,  1.4809e-02,  7.1586e-02,  ..., -3.3472e-01,
                         -4.2204e-01, -2.5897e-01],
                        [ 2.9952e-02,  4.0168e-02,  6.1760e-02,  ...,  4.1205e-01,
                          3.9216e-01,  1.6498e-01],
                        [-1.4526e-02, -4.5165e-03, -2.5143e-02,  ..., -1.5194e-01,
                         -8.3467e-02, -6.8610e-03]],
              
                       [[-1.4098e-02, -2.9452e-02, -3.7444e-02,  ...,  2.9853e-02,
                         -2.0760

In [75]:
# Let's check that the code works if our state dictionary is no longer in memory, so that we have to load it from a .pth file.
instance = resnet18mc_test
print_header("Code test: inference --- model loaded from file")
instance.state_dict = None
display(instance.inference(filename = instance._filename))


CODE TEST: INFERENCE --- MODEL LOADED FROM FILE

image_id, label, ohe-label: ISIC_0027419, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0025030, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033482, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033201, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0032997, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033529, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0034303, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0034031, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033124, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033449, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033676, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0027652, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0031168, 0, tensor([1., 0., 0

image_id, label, ohe-label: ISIC_0030465, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0024943, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0024415, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0032646, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0033627, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0030842, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0029760, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0030555, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0031372, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0026313, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0034135, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033626, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0033780, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_00300

,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_bcc,prob_nv,prob_akiec
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta,0.004196,0.316732,0.368565,0.122264,0.188242
1,HAM_0000118,2,ISIC_0025030,bkl,0,histo,80.0,male,scalp,t1,0.00288,0.479976,0.277723,0.125247,0.114174
2,HAM_0006663,2,ISIC_0033482,bkl,0,histo,85.0,female,lower extremity,t1,0.000856,0.059555,0.761335,0.02119,0.157064
3,HAM_0006663,2,ISIC_0033201,bkl,0,histo,85.0,female,lower extremity,ta,0.007559,0.244466,0.542315,0.080307,0.125353
4,HAM_0003162,2,ISIC_0032997,bkl,0,histo,75.0,male,back,t1,0.042981,0.183496,0.469779,0.192501,0.111242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,HAM_0000967,1,ISIC_0025539,akiec,4,histo,75.0,male,lower extremity,v1,0.034926,0.060543,0.518724,0.137752,0.248054
140,HAM_0005480,2,ISIC_0026149,akiec,4,histo,65.0,male,face,va,0.005054,0.187896,0.277172,0.273446,0.256433
141,HAM_0005480,2,ISIC_0025992,akiec,4,histo,65.0,male,face,v1,0.002191,0.313825,0.264552,0.096804,0.322628
142,HAM_0004592,2,ISIC_0030730,akiec,4,histo,60.0,female,face,v1,0.003655,0.386279,0.402175,0.142765,0.065125


In [324]:
import importlib

import multiclass_models

importlib.reload(multiclass_models)

<module 'multiclass_models' from 'D:\\projects\\skin-lesion-classification\\scripts\\multiclass_models.py'>

In [81]:
from collections import OrderedDict
from multiclass_models import threshold, get_argmax, append_prediction, df_with_probabilities, mode_with_random, predictions_mode

instance = resnet18mc_test

# try:
#     instance._inference_df = df_with_probabilities(instance)
# except:
filename = instance._filename + "_infer.csv" # or whatever it might be
file_path_csv = instance.model_dir.joinpath(filename)
file_path_csv
instance._inference_df = df_with_probabilities(file_path_csv)

print_header("Code test: probabilities for all images".upper())

display(instance._inference_df)

threshold_dict_help = OrderedDict([('mel',0.4)])
threshold_dict_hinder = OrderedDict([('nv',0.6)])

inverse_label_codes = {value: key for key, value in instance.label_codes.items()}

print_header("Code test: probabilities and predictions for all images".upper())
instance._predictions_df = append_prediction(original_df=instance._inference_df, 
                                             probabilities_df=instance._inference_df, 
                                             threshold_dict_help=threshold_dict_help,
                                             threshold_dict_hinder=threshold_dict_hinder,
                                             inverse_label_codes=inverse_label_codes,)

display(instance._predictions_df)

print_header("Code test: probabilities and predictions for images in validation set".upper())
instance._predictions_df_val = instance._predictions_df[instance._predictions_df["set"] == "v1"]

display(instance._predictions_df_val)

print_header("Code test: probabilities and aggregated [...] predictions for images in validation set".upper())
display(predictions_mode(instance._predictions_df))


CODE TEST: PROBABILITIES FOR ALL IMAGES



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_bcc,prob_nv,prob_akiec
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta,0.002810,0.488154,0.320958,0.112851,0.075228
1,HAM_0000118,2,ISIC_0025030,bkl,0,histo,80.0,male,scalp,t1,0.006403,0.546590,0.207624,0.160286,0.079096
2,HAM_0006663,2,ISIC_0033482,bkl,0,histo,85.0,female,lower extremity,t1,0.001759,0.126734,0.534769,0.037566,0.299172
3,HAM_0006663,2,ISIC_0033201,bkl,0,histo,85.0,female,lower extremity,ta,0.005804,0.150670,0.624709,0.059166,0.159651
4,HAM_0003162,2,ISIC_0032997,bkl,0,histo,75.0,male,back,t1,0.058898,0.161691,0.408894,0.197394,0.173124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,HAM_0000967,1,ISIC_0025539,akiec,4,histo,75.0,male,lower extremity,v1,0.109885,0.118041,0.390637,0.202687,0.178750
140,HAM_0005480,2,ISIC_0026149,akiec,4,histo,65.0,male,face,va,0.013788,0.298523,0.205440,0.246306,0.235942
141,HAM_0005480,2,ISIC_0025992,akiec,4,histo,65.0,male,face,v1,0.001394,0.167860,0.157719,0.106153,0.566874
142,HAM_0004592,2,ISIC_0030730,akiec,4,histo,60.0,female,face,v1,0.006594,0.468394,0.284184,0.187424,0.053403



CODE TEST: PROBABILITIES AND PREDICTIONS FOR ALL IMAGES



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_bcc,prob_nv,prob_akiec,pred
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta,0.002810,0.488154,0.320958,0.112851,0.075228,1
1,HAM_0000118,2,ISIC_0025030,bkl,0,histo,80.0,male,scalp,t1,0.006403,0.546590,0.207624,0.160286,0.079096,1
2,HAM_0006663,2,ISIC_0033482,bkl,0,histo,85.0,female,lower extremity,t1,0.001759,0.126734,0.534769,0.037566,0.299172,2
3,HAM_0006663,2,ISIC_0033201,bkl,0,histo,85.0,female,lower extremity,ta,0.005804,0.150670,0.624709,0.059166,0.159651,2
4,HAM_0003162,2,ISIC_0032997,bkl,0,histo,75.0,male,back,t1,0.058898,0.161691,0.408894,0.197394,0.173124,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,HAM_0000967,1,ISIC_0025539,akiec,4,histo,75.0,male,lower extremity,v1,0.109885,0.118041,0.390637,0.202687,0.178750,2
140,HAM_0005480,2,ISIC_0026149,akiec,4,histo,65.0,male,face,va,0.013788,0.298523,0.205440,0.246306,0.235942,1
141,HAM_0005480,2,ISIC_0025992,akiec,4,histo,65.0,male,face,v1,0.001394,0.167860,0.157719,0.106153,0.566874,4
142,HAM_0004592,2,ISIC_0030730,akiec,4,histo,60.0,female,face,v1,0.006594,0.468394,0.284184,0.187424,0.053403,1



CODE TEST: PROBABILITIES AND PREDICTIONS FOR IMAGES IN VALIDATION SET



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_bcc,prob_nv,prob_akiec,pred
109,HAM_0006801,3,ISIC_0032570,bkl,0,histo,60.0,female,lower extremity,v1,0.046393,0.083066,0.102322,0.734325,0.033894,3
110,HAM_0002335,2,ISIC_0031812,bkl,0,histo,85.0,male,upper extremity,v1,0.043069,0.150440,0.688703,0.109991,0.007796,2
112,HAM_0006988,1,ISIC_0024943,bkl,0,consensus,75.0,female,face,v1,0.000012,0.003398,0.116960,0.000692,0.878939,4
113,HAM_0005586,1,ISIC_0024415,nv,3,follow_up,45.0,female,back,v1,0.102728,0.366121,0.024346,0.502185,0.004621,1
115,HAM_0001076,2,ISIC_0033627,nv,3,histo,65.0,male,back,v1,0.423836,0.146423,0.157589,0.152320,0.119832,0
116,HAM_0006855,1,ISIC_0030842,nv,3,histo,50.0,male,back,v1,0.003835,0.209552,0.454396,0.091031,0.241185,2
118,HAM_0004330,2,ISIC_0030555,df,0,histo,70.0,male,lower extremity,v1,0.435236,0.063799,0.146160,0.350733,0.004073,0
120,HAM_0007418,2,ISIC_0026313,df,0,consensus,50.0,male,lower extremity,v1,0.014638,0.335821,0.276991,0.305636,0.066914,1
121,HAM_0006371,3,ISIC_0034135,df,0,consensus,35.0,female,lower extremity,v1,0.018797,0.126853,0.207847,0.636889,0.009614,3
124,HAM_0004126,2,ISIC_0030060,mel,1,histo,80.0,female,back,v1,0.236065,0.137967,0.447473,0.084833,0.093662,2



CODE TEST: PROBABILITIES AND AGGREGATED [...] PREDICTIONS FOR IMAGES IN VALIDATION SET



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_bcc,prob_nv,prob_akiec,pred_mode
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta,0.002810,0.488154,0.320958,0.112851,0.075228,1
1,HAM_0000118,2,ISIC_0025030,bkl,0,histo,80.0,male,scalp,t1,0.006403,0.546590,0.207624,0.160286,0.079096,1
2,HAM_0006663,2,ISIC_0033482,bkl,0,histo,85.0,female,lower extremity,t1,0.001759,0.126734,0.534769,0.037566,0.299172,2
3,HAM_0006663,2,ISIC_0033201,bkl,0,histo,85.0,female,lower extremity,ta,0.005804,0.150670,0.624709,0.059166,0.159651,2
4,HAM_0003162,2,ISIC_0032997,bkl,0,histo,75.0,male,back,t1,0.058898,0.161691,0.408894,0.197394,0.173124,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,HAM_0000967,1,ISIC_0025539,akiec,4,histo,75.0,male,lower extremity,v1,0.109885,0.118041,0.390637,0.202687,0.178750,2
140,HAM_0005480,2,ISIC_0026149,akiec,4,histo,65.0,male,face,va,0.013788,0.298523,0.205440,0.246306,0.235942,1
141,HAM_0005480,2,ISIC_0025992,akiec,4,histo,65.0,male,face,v1,0.001394,0.167860,0.157719,0.106153,0.566874,1
142,HAM_0004592,2,ISIC_0030730,akiec,4,histo,60.0,female,face,v1,0.006594,0.468394,0.284184,0.187424,0.053403,1


In [77]:
predictions_mode(instance._predictions_df)

,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_bcc,prob_nv,prob_akiec,pred_mode
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta,0.002810,0.488154,0.320958,0.112851,0.075228,1
1,HAM_0000118,2,ISIC_0025030,bkl,0,histo,80.0,male,scalp,t1,0.006403,0.546590,0.207624,0.160286,0.079096,1
2,HAM_0006663,2,ISIC_0033482,bkl,0,histo,85.0,female,lower extremity,t1,0.001759,0.126734,0.534769,0.037566,0.299172,2
3,HAM_0006663,2,ISIC_0033201,bkl,0,histo,85.0,female,lower extremity,ta,0.005804,0.150670,0.624709,0.059166,0.159651,2
4,HAM_0003162,2,ISIC_0032997,bkl,0,histo,75.0,male,back,t1,0.058898,0.161691,0.408894,0.197394,0.173124,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,HAM_0000967,1,ISIC_0025539,akiec,4,histo,75.0,male,lower extremity,v1,0.109885,0.118041,0.390637,0.202687,0.178750,2
140,HAM_0005480,2,ISIC_0026149,akiec,4,histo,65.0,male,face,va,0.013788,0.298523,0.205440,0.246306,0.235942,1
141,HAM_0005480,2,ISIC_0025992,akiec,4,histo,65.0,male,face,v1,0.001394,0.167860,0.157719,0.106153,0.566874,1
142,HAM_0004592,2,ISIC_0030730,akiec,4,histo,60.0,female,face,v1,0.006594,0.468394,0.284184,0.187424,0.053403,1


In [79]:
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = resnet18mc_test
map_labels = instance.label_codes

A = instance._predictions_df_val['label']
B = instance._predictions_df_val['pred']
AxB = pd.crosstab(A,B,margins=True,dropna=False)

beta = 1
weights = None

instance._cm = confusion_matrix_with_metric(AxB=AxB,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Code test: confusion matrix (validation set)")

to_print = ["- The overall evaluation metric would appear at the bottom right, if it were defined (this code test set is too small).",
            "It would be a class-wise weighted average fbeta score, beta and weights as specified (default values 1).",
            "One could also pass None, a float, or a function other than weighted_average_f to the func parameter in confusion_matrix_with_metric."]
print("\n- ".join(to_print))
display(instance._cm.fillna('_'))


CODE TEST: CONFUSION MATRIX (VALIDATION SET)

- The overall evaluation metric would appear at the bottom right, if it were defined (this code test set is too small).
- It would be a class-wise weighted average fbeta score, beta and weights as specified (default values 1).
- One could also pass None, a float, or a function other than weighted_average_f to the func parameter in confusion_matrix_with_metric.


predicted,other,mel,bcc,nv,akiec,All,recall
actual,,,,,,,
other,3,1,2,2,1,9,0.333333
mel,0,0,2,0,1,3,0.0
bcc,0,0,3,0,0,3,1.0
nv,1,1,1,0,0,3,0.0
akiec,0,1,1,0,1,3,0.333333
All,4,3,9,2,3,21,_
precision,0.75,0.0,0.333333,0.0,0.333333,_,_


In [80]:
from evaluation import metric_dictionary
# import numpy as np
# import pandas as pd

instance = resnet18mc_test

target = instance._predictions_df['label'] 
prediction = instance._predictions_df['pred'] 
probabilities = instance._predictions_df.filter(regex=r'^prob_')

# target = pd.concat([target_, pd.Series([1,2])])
# target = target.reset_index(drop=True)
# prediction = pd.concat([prediction_, pd.Series([0,2])]) 
# prediction = prediction.reset_index(drop=True)
# probabilities = probabilities_.copy()
# probabilities.loc[16] = 0.2
# probabilities.loc[17] = 0.2
# probabilities = probabilities.values

print_header("Code test: other metrics")
to_print = ["- ACC: accuracy",
            "BACC: balanced accuracy",
            "precision: macro-averaged precision (equal weight to each class)",
            "recal: macro-averaged recall (equal weight to each class)",
            "Fbeta: macro-averaged F_beta score (equal weight to each class)",
            "MCC: Matthews correlation coefficient",
            "ROC-AUC mac: macro-averaged ROC-AUC (equal weight to each class)",
            "ROC-AUC wt: weighted-average ROC-AUC (larger class -> more weight)",
            "ROC-AUC wt*: weighted-average ROC-AUC (larger class -> *less weight)",            
            ]

instance._metric_dict = metric_dictionary(target=target, 
                                          prediction=prediction, 
                                          probabilities=probabilities)
print("\n- ".join(to_print))
display(pd.DataFrame(instance._metric_dict))


CODE TEST: OTHER METRICS

- ACC: accuracy
- BACC: balanced accuracy
- precision: macro-averaged precision (equal weight to each class)
- recal: macro-averaged recall (equal weight to each class)
- Fbeta: macro-averaged F_beta score (equal weight to each class)
- MCC: Matthews correlation coefficient
- ROC-AUC mac: macro-averaged ROC-AUC (equal weight to each class)
- ROC-AUC wt: weighted-average ROC-AUC (larger class -> more weight)
- ROC-AUC wt*: weighted-average ROC-AUC (larger class -> *less weight)


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.277778,0.350072,0.346472,0.350072,0.283516,0.271494,0.298296,0.158465,0.722044,0.72177,0.726919


### Baseline model

In [22]:
# import torchvision.transforms as transforms

transform = transforms.Compose([
    
transforms.RandomCrop((300, 300)),
transforms.Resize((224,224)), # Resize images to fit ResNet input size
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])    

In [23]:
# import pandas as pd
# from typing import List, Callable

df: pd.DataFrame = metadata.df                   # Background dataset for the model. metadata._df_sample_batch is a random selection of 64 rows of metadata.df. We use it for testing our code.
train_set: Union[pd.DataFrame, list, str] = "t1" # "t1" (one image per lesion in training set); ["t1", "ta"] (all images for each lesion in training set); can also specify another sub-dataframe of self.df.
val_set: Union[pd.DataFrame, list, str] = "v1"   # Similar to train_set above.
label_codes: dict = metadata._label_codes        # Correspondence between label codes like 0 and label words like 'other'.
data_dir: Path = path["images"]                  # Path to directory where images are stored.
model_dir: Path = path["models"]                 # Path to directory where models/model info/model results are stored.
transform: List[Callable] = transform            # Transform to be applied to images before feeding to ResNet-18
batch_size: int = 32                             # Mini-batch size: default 32.
epochs: int = 10                                 # Number of epochs (all layers unfrozen from the start): default 10.
base_learning_rate: float = 0.001                # Learning rate to start with: default 0.001. Using Adam optimizer.
filename_stem: str = "rn18mc"                    # For saving model and related files. train set and num epochs will be appended automatically. Default "rn18mc".
filename_suffix: str = "base"                    # Something descriptive and unique for future reference and to avoid over-writing other files. Default empty string "".

In [24]:
resnet18mc_base = resnet18(                                  
    df=df, 
    train_set=train_set,
    val_set=val_set,
    label_codes=label_codes,
    data_dir=data_dir,
    model_dir=model_dir,
    transform=transform,
    batch_size=batch_size,
    epochs=epochs,                                                
    base_learning_rate=base_learning_rate,
    filename_stem=filename_stem,
    filename_suffix=filename_suffix,                                  
)

In [ ]:
# We only need to do this once...
# resnet18mc_base.train()

In [404]:
import importlib

del instance._inference_df
import evaluation

importlib.reload(evaluation)

<module 'evaluation' from 'D:\\projects\\skin-lesion-classification\\scripts\\evaluation.py'>

In [413]:
# from evaluation import get_argmax, append_prediction, get_probabilities_df

instance = resnet18mc_base

# try:
#     instance._inference_df = df_with_probabilities(instance)
# except:
filename = "rn18mc_t1_10e_base_inference.csv" #instance._filename + "_infer.csv" # "rn18mc_t1_10e_base_inference.csv"
file_path_csv = instance.model_dir.joinpath(filename)
instance._inference_df = df_with_probabilities(file_path_csv)
instance._inference_df = pd.merge(instance.df, instance._inference_df, on='image_id', how='inner')

print_header("Baseline model: probabilities for all images".upper())

display(instance._inference_df.head())

inverse_label_codes = {value: key for key, value in instance.label_codes.items()}

print_header("Baseline model: probabilities and predictions for all images".upper())
instance._predictions_df = append_prediction(original_df=instance._inference_df, 
                                        probabilities_df=instance._inference_df, 
                                        inverse_label_codes=inverse_label_codes,)

display(instance._predictions_df.head())

print_header("Baseline model: probabilities and predictions for images in validation set".upper())
instance._predictions_df_val = instance._predictions_df[instance._predictions_df["set"] == "v1"]

display(instance._predictions_df_val.head())


BASELINE MODEL: PROBABILITIES FOR ALL IMAGES



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_akiec,prob_nv,prob_bcc
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta,0.190499,0.589043,0.070307,0.127727,0.022423
1,HAM_0000118,2,ISIC_0025030,bkl,0,histo,80.0,male,scalp,t1,0.325244,0.116826,0.034226,0.438181,0.085523
2,HAM_0002730,2,ISIC_0026769,bkl,0,histo,80.0,male,scalp,va,0.025946,0.010540,0.299041,0.045798,0.618675
3,HAM_0002730,2,ISIC_0025661,bkl,0,histo,80.0,male,scalp,v1,0.039586,0.041373,0.251411,0.027629,0.640001
4,HAM_0001466,2,ISIC_0031633,bkl,0,histo,75.0,male,ear,va,0.110923,0.402103,0.215037,0.059775,0.212162



BASELINE MODEL: PROBABILITIES AND PREDICTIONS FOR ALL IMAGES



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_akiec,prob_nv,prob_bcc,pred
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta,0.190499,0.589043,0.070307,0.127727,0.022423,2
1,HAM_0000118,2,ISIC_0025030,bkl,0,histo,80.0,male,scalp,t1,0.325244,0.116826,0.034226,0.438181,0.085523,4
2,HAM_0002730,2,ISIC_0026769,bkl,0,histo,80.0,male,scalp,va,0.025946,0.010540,0.299041,0.045798,0.618675,1
3,HAM_0002730,2,ISIC_0025661,bkl,0,histo,80.0,male,scalp,v1,0.039586,0.041373,0.251411,0.027629,0.640001,1
4,HAM_0001466,2,ISIC_0031633,bkl,0,histo,75.0,male,ear,va,0.110923,0.402103,0.215037,0.059775,0.212162,2



BASELINE MODEL: PROBABILITIES AND PREDICTIONS FOR IMAGES IN VALIDATION SET



,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_akiec,prob_nv,prob_bcc,pred
3,HAM_0002730,2,ISIC_0025661,bkl,0,histo,80.0,male,scalp,v1,0.039586,0.041373,0.251411,0.027629,0.640001,1
5,HAM_0001466,2,ISIC_0027850,bkl,0,histo,75.0,male,ear,v1,0.118692,0.143967,0.113587,0.164686,0.459067,1
7,HAM_0002761,2,ISIC_0029068,bkl,0,histo,60.0,male,face,v1,0.041351,0.002828,0.025426,0.131957,0.798437,1
11,HAM_0004234,2,ISIC_0029396,bkl,0,histo,85.0,female,chest,v1,0.045510,0.151270,0.007371,0.789996,0.005853,4
13,HAM_0001949,2,ISIC_0025767,bkl,0,histo,70.0,male,trunk,v1,0.712821,0.006769,0.000950,0.272685,0.006774,0


In [428]:
# from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = resnet18mc_base
map_labels = instance.label_codes

A = instance._predictions_df_val['label']
B = instance._predictions_df_val['pred']
AxB = pd.crosstab(A,B,margins=True,dropna=False)

beta = 2

instance._cm = confusion_matrix_with_metric(AxB=AxB,
                                                lst=None,
                                                full_pad=True,
                                                func=weighted_average_f,
                                                beta=beta,
                                                weights=None,
                                                percentage=False,
                                                map_labels=map_labels)

print_header("BASELINE MODEL: confusion matrix (validation set)")

print(f"Overall evaluation metric at bottom right: average class-wise F{beta} score".upper())
display(instance._cm.fillna('_'))

C = instance._predictions_df['label']
D = instance._predictions_df['pred']
CxD = pd.crosstab(C,D,margins=True,dropna=False)

confusion_matrix_with_metric(AxB=CxD,
                                                lst=None,
                                                full_pad=True,
                                                func=weighted_average_f,
                                                beta=beta,
                                                weights=None,
                                                percentage=False,
                                                map_labels=map_labels)


BASELINE MODEL: CONFUSION MATRIX (VALIDATION SET)

OVERALL EVALUATION METRIC AT BOTTOM RIGHT: AVERAGE CLASS-WISE F2 SCORE


predicted,other,bcc,mel,akiec,nv,All,recall
actual,,,,,,,
other,78,19,46,11,71,225,0.346667
bcc,5,46,4,9,18,82,0.560976
mel,9,2,65,9,69,154,0.422078
akiec,7,20,6,20,4,57,0.350877
nv,35,9,43,5,"1,259","1,351",0.931902
All,134,96,164,54,"1,421","1,869",_
precision,0.58209,0.479167,0.396341,0.37037,0.885996,_,0.52265


predicted,other,bcc,mel,akiec,nv,All,recall
actual,,,,,,,
other,612,86,312,53,293,"1,356",0.451327
bcc,14,367,26,21,86,514,0.714008
mel,45,7,587,36,438,"1,113",0.527403
akiec,19,70,14,200,24,327,0.611621
nv,184,54,278,19,"6,170","6,705",0.920209
All,874,584,"1,217",329,"7,011","10,015",NaN
precision,0.700229,0.628425,0.482334,0.607903,0.880046,NaN,0.644286
